## 기초 분류 모델 만들기

## 라이브러리 로드


In [ ]:
# 필요 라이브러리 설치
# koreanize-matplotlib : 한글폰트 사용을 위해 
# konlpy : 한국어 형태소 분석을 위해 
# tqdm : 오래 걸리는 작업의 진행상태를 보기 위해 
# !pip install koreanize-matplotlib
# !pip install konlpy --upgrade
# !pip install tqdm --upgrade

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 시각화를 위한 폰트 설정


In [ ]:
import koreanize_matplotlib

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

pd.Series([1, 3, 5, -7, 9]).plot(title="한글")

## 데이터 로드

In [ ]:
# 데이콘의 해당 데이터셋은 CC-BY-4.0 라이센스입니다.
# 데이터 출처 : https://dacon.io/competitions/official/235747/data
# 로컬 PC에서 실습 시 직접 데이콘 사이트에 회원가입하고 다운로드 해주세요.

import os, platform

base_path = "data/klue/"
file_name = "dacon-klue-open-zip"

def file_exist_check(base_path):
    if os.path.exists(f"{base_path}train_data.csv"):
        print(f"{os.getcwd()}/{base_path} 경로에 파일이 있음")
        return

    if not os.path.exists(base_path):
        os.makedirs(base_path)
    
    if platform.system() == "Linux":
        print(f"파일을 다운로드 하고 {base_path} 경로에 압축을 해제함")
        !wget https://bit.ly/{file_name}
        !unzip {file_name} -d {base_path}
        return 
    else:
        print(f"""https://dacon.io/competitions/official/235747/data 에서 다운로드 하고
              실습 경로 {os.getcwd()}/{base_path}에 옮겨주세요.""")
        return
    
file_exist_check(base_path) 

In [ ]:
# 학습, 예측 데이터셋을 불러옵니다.
train = pd.read_csv("data/klue/train_data.csv")
test = pd.read_csv("data/klue/test_data.csv")
train.shape, test.shape

In [ ]:
# 토픽을 불러옵니다.
topic = pd.read_csv("data/klue/topic_dict.csv")
topic

In [ ]:
train.head()

In [ ]:
test.head()

## 전처리를 위한 데이터 병합

In [ ]:
topic["topic"].values

In [ ]:
# 전처리를 위해 데이터 병합
raw = pd.concat([train, test])
raw.shape

In [ ]:
raw.head()

In [ ]:
raw.tail()

In [ ]:
df = raw.merge(topic, how="left")
df.shape

In [ ]:
df.head()

## 정답값 빈도수

In [ ]:
# test는 결측치로 되어 있기 때문에 빈도수에 포함되지 않습니다.
df["topic_idx"].value_counts()

In [ ]:
# df 로 빈도수를 구했지만 test 데이터는 topic이 결측치라 포함되지 않습니다. 
sns.countplot(data=df, y="topic")

## 문자 길이

In [ ]:
# 문자, 단어 빈도수 파생변수 만들기
df["len"] = df["title"].apply(lambda x : len(x))
df["word_count"] = df["title"].apply(lambda x : len(x.split()))
df["unique_word_count"] = df["title"].apply(lambda x : len(set(x.split())))

In [ ]:
# 파생변수가 잘 만들어졌는지 확인하기
df.head()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 2))
sns.histplot(df["len"], ax=axes[0])
sns.histplot(df["word_count"], ax=axes[1])
sns.histplot(df["unique_word_count"], ax=axes[2])

In [ ]:
df[["len", "word_count", "unique_word_count"]].describe()

### 주제별 글자와 단어 수

In [ ]:
sns.displot(data=df, x="len",
            hue="topic", col="topic", col_wrap=2, aspect=5, height=2)

In [ ]:
sns.displot(data=df, x="word_count",
            hue="topic", col="topic", col_wrap=2, aspect=5, height=2)

In [ ]:
sns.displot(data=df, x="unique_word_count",
            hue="topic", col="topic", col_wrap=2, aspect=5, height=2)

## 문자 전처리

### 숫자 제거

In [ ]:
import re
# df["title"] = df["title"].map(lambda x : re.sub("[0-9]", "", x))
df["title"] = df["title"].str.replace("[0-9]", "", regex=True)

### 영문자는 모두 소문자로 변경

대소문자가 섞여 있으면 다른 다른 단어로 다루기 때문에 영문자는 모두 대문자 혹은 소문자로 변경합니다.

In [ ]:
df["title"] = df["title"].str.lower()

### 조사, 어미, 구두점 제거

In [ ]:
# 형태소 분석기(Okt) 불러오기 

from konlpy.tag import Okt
okt = Okt() 

# 조사, 어미, 구두점 제거, 어간 추출
def okt_clean(text):
    clean_text = []
    for word in okt.pos(text, stem=True):
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']:
            clean_text.append(word[0])
    
    return " ".join(clean_text) 

from tqdm import tqdm
tqdm.pandas() 

train['title'] = train['title'].progress_map(okt_clean)
test['title'] = test['title'].progress_map(okt_clean)

### 불용어 제거

In [ ]:
# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = [ '합니다', '하는', '할', '하고', '한다', 
             '그리고', '입니다', '그 ', ' 등', '이런', ' 것 ', ' 및 ',' 제 ', ' 더 ']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words)

In [ ]:
df["title"] = df["title"].map(remove_stopwords)

## 학습, 예측 데이터셋 분리

In [ ]:
label_name = "topic_idx"

In [ ]:
train = df[df[label_name].notnull()]
test = df[df[label_name].isnull()]
train.shape, test.shape

In [ ]:
X_train = train["title"]
X_test = test["title"]

X_train.shape, X_test.shape

In [ ]:
y_train = train[label_name]
y_train.value_counts()

In [ ]:
y_test = test[label_name]
y_test.value_counts()

## 벡터화
* 머신러닝이나 딥러닝 알고리즘은 문자를 이해할 수 없습니다. 내부에서는 수치 계산이 이루어지기 때문에 문자를 숫자로 변경해 주어야 합니다.


### TF-IDF(Term Frequency - Inverse Document Frequency)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=None, 
                             ngram_range=(1,2),
                             min_df=3, 
                             max_df=0.95)
tfidf_vect.fit(X_train)

In [ ]:
train_feature_tfidf = tfidf_vect.transform(X_train)
test_feature_tfidf = tfidf_vect.transform(X_test)

train_feature_tfidf.shape, test_feature_tfidf.shape

In [ ]:
# 단어 사전
vocab = tfidf_vect.get_feature_names()
print(len(vocab))
vocab[:10]

In [ ]:
# np.sum 으로 위에서 구한 train_feature_vector 의 값을 모두 더합니다. axis=0 으로 합니다. 
dist = np.sum(train_feature_tfidf, axis=0)

vocab_count = pd.DataFrame(dist, columns=vocab)
vocab_count

In [ ]:
# 위에서 구한 빈도수를 그래프로 그립니다.
vocab_count.T[0].sort_values(ascending=False).head(50).plot.bar(figsize=(15, 4))

## 학습과 예측

In [ ]:
# RandomForestClassifier 를 불러옵니다.
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
model = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state=42)
model

In [ ]:
from lightgbm import LGBMClassifier

# lightgbm 분류기 사용
model = LGBMClassifier(random_state=42)
model

### 교차 검증

In [ ]:
from sklearn.model_selection import cross_val_predict

y_pred = cross_val_predict(model, train_feature_tfidf, y_train, cv=3, n_jobs=-1, verbose=1)

### 교차 검증 정확도

In [ ]:
valid_accuracy = (y_pred == y_train).mean()
valid_accuracy

In [ ]:
df_accuracy = pd.DataFrame({"pred": y_pred, "train": y_train})
df_accuracy["accuracy"] = (y_pred == y_train)

In [ ]:
topic

In [ ]:
df_accuracy.groupby(["train"])["accuracy"].mean()

In [ ]:
df_accuracy.rename(columns={"pred":"predict"})

### 학습


In [ ]:
# fit 으로 학습시킵니다.
%time model.fit(train_feature_tfidf, y_train)

### 예측

In [ ]:
# predict로 예측합니다. 
y_predict = model.predict(test_feature_tfidf)
y_predict[:5]

## 답안지 로드

sample_submission.csv 파일은 마치 답안지와 같습니다.

In [ ]:
submit = pd.read_csv("data/klue/sample_submission.csv")
submit.head()

In [ ]:
# 정답값 측정을 위해 y_test 변수에 할당
submit["topic_idx"] = y_predict

In [ ]:
submit.to_csv(f"submit_{valid_accuracy}.csv", index=False)